In [9]:
using CSV, DataFrames, Statistics

In [10]:
prefix = "aistats-rev/"

"aistats-rev/"

In [11]:
function method_category(meth)
    if startswith(meth, "Imp-then-Reg")
        return "Imp-then-Reg"
    elseif startswith(meth, "Joint Imp-then-Reg")
        return "Joint Imp-then-Reg"
    elseif meth ∈ ["Static", "Affine", "Finite"]
        return "Adaptive LR"
    elseif startswith(meth, "Complete Features")
        return "Complete Features"
    else 
        return meth
    end
end

method_category (generic function with 1 method)

In [12]:
pb_datasets = ["cylinder-bands", "ozone-level-detection-eight", "ozone-level-detection-one", "thyroid-disease-thyroid-0387", "trains",
                "credit-approval", "Ecdat-Mofa", "sleep"]

8-element Vector{String}:
 "cylinder-bands"
 "ozone-level-detection-eight"
 "ozone-level-detection-one"
 "thyroid-disease-thyroid-0387"
 "trains"
 "credit-approval"
 "Ecdat-Mofa"
 "sleep"

## For Real X - Syn Y Experiments

In [13]:
prefix ="aistats-rev/"
setting = prefix*"fakey/"
for y_model in ["linear", "nn"]
    for m_model = ["mar", "nmar", "mar_adv"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
        
        filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv")]
        res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)
        
        for subdir = ["all/"]
            filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv")]
#             res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
            for i in 1:length(filelist)
                res = vcat(res, CSV.read(directory*subdir*filelist[i], DataFrame))
            end
        end

    #     for subdir = ["xgb/"]
    #         try 
    #             filelist = [f for f in readdir("xgboost/"*directory*subdir) if endswith(f, ".csv")]
    #             for i in 1:length(filelist)
    #                 res = vcat(res, CSV.read("xgboost/"*directory*subdir*filelist[i], DataFrame))
    #             end
    #         catch 
    #             println("No XGBoost results for "*directory*subdir)
    #         end
    #     end

        res[!,:method_cat] = map(t -> method_category(t), res[:,:method])
        res[!,:X_setting] .= "real_X_"*m_model
        res[!,:Y_setting] .= "syn_Y_"*y_model

        CSV.write(directory*"FINAL_results.csv", res)
    end
end

In [14]:
setting = prefix*"fakey/"

for y_model in ["linear", "nn"]
    for m_model = ["mar", "nmar", "mar_adv"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
  
        res = CSV.read(directory*"FINAL_results.csv", DataFrame)
        
        res[!,:method] .= map(t -> (t == "Affine" ? "Adaptive LR - Affine" : t), res[:,:method]) 
        res[!,:method] .= map(t -> (t == "Finite" ? "Adaptive LR - Finite" : t), res[:,:method]) 
        res[!,:method] .= map(t -> (t == "Static" ? "Adaptive LR - Affine intercept only" : t), res[:,:method])
        
        filter!(t -> t[:dataset] ∉ pb_datasets, res)

        for method in ["Oracle X", "Oracle XM", "Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Imp-then-Reg 5", "Joint Imp-then-Reg", "Adaptive LR"]
            aux = filter(t -> startswith(t[:method], method), res)

            idcols = [:dataset, :X_setting, :Y_setting, :SNR, :k, :kMissing, :splitnum]
            gd = groupby(aux, idcols)

            aux = similar(aux, 0)
            for subdf in gd 
                scoremax = argmax(subdf[:,:score])
                push!(aux, subdf[scoremax,names(aux)])
            end
            aux[!,:method] .= method*" - best"

            res = vcat(res, aux)
        end
        
        CSV.write(directory*"FINAL_results.csv", res)
    end
end

Sanity check

In [46]:
res = DataFrames.similar(CSV.read(prefix*"fakey/"*"linear"*"_"*"mar"*"/FINAL_results.csv", DataFrame), 0)
for y_model in ["linear", "nn"]
    for m_model = ["mar", "nmar", "mar_adv"]
        dir = y_model*"_"*m_model*"/"
        directory = prefix*"fakey/"*dir
  
        res = vcat(res, CSV.read(directory*"FINAL_results.csv", DataFrame))
    end
end

In [47]:
res

Row,dataset,SNR,k,kMissing,splitnum,method,r2,osr2,r2list,osr2list,time,hp,score,method_cat,X_setting,Y_setting
,String,Int64,Int64,Int64,Int64,String,Float64,Float64,String,String,Float64,String,Float64,String31,String15,String15
1,COUNT-loomis,2,10,0,1,XGBoost,0.737051,0.742668,Any[0.7370507983003595],Any[0.74266763420168],23.4184,"Dict{Symbol, Real}(:min_child_weight => 6, :gamma => 0.1, :n_estimators => 50, :max_depth => 3)",0.687844,XGBoost,real_X_mar,syn_Y_linear
2,COUNT-loomis,2,10,0,1,Oracle X - linear,0.73965,0.762056,Any[0.7396500880253674],Any[0.7620558281755224],3.73222,"Dict{Symbol, Any}(:alpha => 0.3, :regtype => :lasso)",0.705147,Oracle X - linear,real_X_mar,syn_Y_linear
3,COUNT-loomis,2,10,0,1,Oracle XM - linear,0.73965,0.762056,Any[0.7396500880253674],Any[0.7620558281755224],0.0932829,"Dict{Symbol, Any}(:alpha => 0.3, :regtype => :lasso)",0.705147,Oracle XM - linear,real_X_mar,syn_Y_linear
4,COUNT-loomis,2,10,0,1,Complete Features - linear,0.73965,0.762056,Any[0.7396500880253674],Any[0.7620558281755224],0.08202,"Dict{Symbol, Any}(:alpha => 0.3, :regtype => :lasso)",0.705147,Complete Features,real_X_mar,syn_Y_linear
5,COUNT-loomis,2,10,0,1,Oracle X - tree,0.769396,0.725498,Any[0.7693959852749067],Any[0.7254980907285085],1.04029,Dict(:maxdepth => 16),0.645335,Oracle X - tree,real_X_mar,syn_Y_linear
6,COUNT-loomis,2,10,0,1,Oracle XM - tree,0.769396,0.725498,Any[0.7693959852749067],Any[0.7254980907285085],0.011651,Dict(:maxdepth => 16),0.645335,Oracle XM - tree,real_X_mar,syn_Y_linear
7,COUNT-loomis,2,10,0,1,Complete Features - tree,0.769396,0.725498,Any[0.7693959852749067],Any[0.7254980907285085],0.0117469,Dict(:maxdepth => 16),0.645335,Complete Features,real_X_mar,syn_Y_linear
8,COUNT-loomis,2,10,0,1,Oracle X - rf,0.740138,0.725082,Any[0.7401381733617556],Any[0.7250819111243514],5.16308,"Dict(:ntrees => 75, :maxdepth => 10)",0.676238,Oracle X - rf,real_X_mar,syn_Y_linear
9,COUNT-loomis,2,10,0,1,Oracle XM - rf,0.740138,0.725082,Any[0.7401381733617556],Any[0.7250819111243514],4.19069,"Dict(:ntrees => 75, :maxdepth => 10)",0.676238,Oracle XM - rf,real_X_mar,syn_Y_linear


In [48]:
gd = groupby(res, [:dataset, :X_setting, :Y_setting, :kMissing, :method])
aggres = combine(gd, nrow)

Row,dataset,X_setting,Y_setting,kMissing,method,nrow
,String,String15,String15,Int64,String,Int64
1,COUNT-loomis,real_X_mar,syn_Y_linear,0,XGBoost,10
2,COUNT-loomis,real_X_mar,syn_Y_linear,0,Oracle X - linear,10
3,COUNT-loomis,real_X_mar,syn_Y_linear,0,Oracle XM - linear,10
4,COUNT-loomis,real_X_mar,syn_Y_linear,0,Complete Features - linear,10
5,COUNT-loomis,real_X_mar,syn_Y_linear,0,Oracle X - tree,10
6,COUNT-loomis,real_X_mar,syn_Y_linear,0,Oracle XM - tree,10
7,COUNT-loomis,real_X_mar,syn_Y_linear,0,Complete Features - tree,10
8,COUNT-loomis,real_X_mar,syn_Y_linear,0,Oracle X - rf,10
9,COUNT-loomis,real_X_mar,syn_Y_linear,0,Oracle XM - rf,10


In [49]:
feasible_combinations = unique(aggres[:,[:dataset, :kMissing]])
feasible_combinations = crossjoin(
                            crossjoin(
                                crossjoin(feasible_combinations, DataFrame(splitnum=1:10)),
                                DataFrame(X_setting = ["real_X_mar", "real_X_nmar", "real_X_mar_adv"])), 
                                DataFrame(Y_setting = ["syn_Y_linear", "syn_Y_nn"])
                            )

pbdatasets = antijoin(feasible_combinations, unique(res[:,[:dataset, :kMissing, :splitnum, :X_setting, :Y_setting]]), on=[:dataset, :kMissing, :splitnum, :X_setting, :Y_setting])
pbdatasets = unique(pbdatasets[:,[:dataset, :X_setting, :Y_setting, :splitnum]])

dataset_list = [d for d in readdir("../datasets/") if !startswith(d, ".")]
sort!(dataset_list)

pbdatasets[!,:array_num] .= (pbdatasets[:,:splitnum] .- 1) .* 71
pbdatasets[!,:array_num] .+= map(t -> findfirst(t .== dataset_list)-1, pbdatasets[:,:dataset])

pbdatasets[!,:back_dnum] .= map(t -> dataset_list[mod(t, 71) + 1], pbdatasets[:,:array_num])
pbdatasets[!,:back_splitnum] .= map(t -> div(t, 71) + 1, pbdatasets[:,:array_num])

@assert all(pbdatasets[:,:back_dnum] .== pbdatasets[:,:dataset])
@assert all(pbdatasets[:,:back_splitnum] .== pbdatasets[:,:splitnum])

In [50]:
function list_to_slurmarray(l)
    s = ""
    begin_seq = -1; last_seq = -1; is_sequence = false

    for an in l
        if an == last_seq + 1
            last_seq = an
        else 
            if is_sequence
                if begin_seq == last_seq
                    s *= string(begin_seq)*","
                else
                    s *= string(begin_seq)*"-"*string(last_seq)*","
                end
            end
            begin_seq = an
            last_seq = an
            is_sequence = true
        end
    end 
    s
end

list_to_slurmarray (generic function with 1 method)

In [51]:
for y_model in ["linear", "nn"]
    println(y_model)
    for m_model = ["mar", "nmar", "mar_adv"]
        println(m_model)
        aux = filter(t -> t[:Y_setting] == "syn_Y_"*y_model && t[:X_setting] == "real_X_"*m_model, pbdatasets)
        l = sort(unique(aux[:,:array_num]))
        @show list_to_slurmarray(l)
    end
    println()
end

linear
mar
list_to_slurmarray(l) = "23-24,31,70,94-95,102,115,165-166,173,186,191,205-206,236-237,257,279,328,333,348,376,378,386,416,449,470,490,499,520,541,567,589,591,612,641,662,683,703,"
nmar
list_to_slurmarray(l) = "4,23,42,49,62-63,70,94-95,115,141,146,165-166,186,212,223,236-237,244,257,307,328,354,378,399,425,449-450,457,470,496,520-521,528,541,548,558,572-573,591,612,628,662,670,"
mar_adv
list_to_slurmarray(l) = "23-24,31,44,70,94-95,102,113,115,141,152,165,186,218,234,236,257,264,274,276,279-280,307,315,328,335,346,378-379,428,430,449-450,457,470,499,507,518,520-521,528,541,548,561,563,570,572,591,612,628,641,643,662,"

nn
mar
list_to_slurmarray(l) = "23-24,31,44,70,81,83,94-95,102,115,120,134,147,152,165-166,173,179,186,191,193,203-208,236-237,244,257,276,278,307-308,315,328,333,335,345-347,350,354,365,378-379,386,399,416,419,421-422,425,430,443,449-450,457,464,470,475,477,487-491,493,496,520-521,538,541,546,560-564,567,591-592,599,610,612,617,619,629-635,638,649,652,662-66

## For Real Data Experiments

In [34]:
prefix

"aistats-rev/"

In [35]:
for directory = prefix .* ["realy/"]
    filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
    res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)
    
    # for subdir = ["2022-08-23/", "rf_mia/"]    
    for subdir = ["all/", "itr/"]
        filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
#         res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
        for i in 1:length(filelist)
            res = vcat(res, CSV.read(directory*subdir*filelist[i], DataFrame))
        end
    end

#     for subdir = ["xgb/"]
#         try 
#             filelist = [f for f in readdir("xgboost/"*directory*subdir) if endswith(f, ".csv")]
#             for i in 1:length(filelist)
#                 res = vcat(res, CSV.read("xgboost/"*directory*subdir*filelist[i], DataFrame))
#             end
#         catch 
#             println("No XGBoost results for "*directory*subdir)
#         end
#     end
    
    # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
    res[!,:method_cat] = map(method_category, res[:,:method])
    res[!,:X_setting] .= "real_X"
    res[!,:Y_setting] .= "real_Y"
    
    CSV.write(directory*"FINAL_results.csv", res)
end

Create `best` variant

In [36]:
res = CSV.read(prefix*"realy/"*"FINAL_results.csv", DataFrame)

filter!(t -> t[:dataset] ∉ pb_datasets, res)

res[!,:method] .= map(t -> (t == "Affine" ? "Adaptive LR - Affine" : t), res[:,:method]) 
res[!,:method] .= map(t -> (t == "Finite" ? "Adaptive LR - Finite" : t), res[:,:method]) 
res[!,:method] .= map(t -> (t == "Static" ? "Adaptive LR - Affine intercept only" : t), res[:,:method])

for method in ["Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Imp-then-Reg 5", "Joint Imp-then-Reg", "Adaptive LR"]
    aux = filter(t -> startswith(t[:method], method), res)
    # @show size(aux)
    idcols = [:dataset, :SNR, :k, :kMissing, :splitnum]
    gd = groupby(aux, idcols)

    aux = similar(aux, 0)
    for subdf in gd 
        scoremax = argmax(subdf[:,:score])
        push!(aux, subdf[scoremax,names(aux)])
    end
    aux[!,:method] .= method*" - best"

    res = vcat(res, aux)
end

CSV.write(prefix*"realy/"*"FINAL_results.csv", res)

"aistats-rev/realy/FINAL_results.csv"

Sanity check

In [37]:
res = CSV.read(prefix*"realy/"*"FINAL_results.csv", DataFrame)
unique(filter( t-> t[:nrow] < 10, combine(groupby(res, [:dataset, :method]), nrow))[:,[:dataset, :nrow]])

Row,dataset,nrow
,String,Int64
1,breast-cancer-wisconsin-prognostic,9
2,communities-and-crime-2,4
3,communities-and-crime,9
4,heart-disease-va,9
5,horse-colic,9
6,mammographic-mass,9
7,mlmRev-star,4
8,thyroid-disease-allhypo,7


In [38]:
unique(res[:,:method])

40-element Vector{String}:
 "XGBoost"
 "Complete Features - linear"
 "Complete Features - tree"
 "Complete Features - rf"
 "CART MIA"
 "RF MIA"
 "Imp-then-Reg 1 - xgboost"
 "Imp-then-Reg 2 - xgboost"
 "Imp-then-Reg 3 - xgboost"
 "Imp-then-Reg 4 - xgboost"
 ⋮
 "Imp-then-Reg 5 - rf"
 "Complete Features - best"
 "Imp-then-Reg 1 - best"
 "Imp-then-Reg 2 - best"
 "Imp-then-Reg 3 - best"
 "Imp-then-Reg 4 - best"
 "Imp-then-Reg 5 - best"
 "Joint Imp-then-Reg - best"
 "Adaptive LR - best"

In [39]:
dataset_list = [d for d in readdir("../datasets/") if !startswith(d, ".")]
sort!(dataset_list)

71-element Vector{String}:
 "COUNT-loomis"
 "Ecdat-MCAS"
 "Ecdat-Males"
 "Ecdat-Mofa"
 "Ecdat-RetSchool"
 "Ecdat-Schooling"
 "MASS-Cars93"
 "MASS-Pima.tr2"
 "MASS-survey"
 "Zelig-coalition2"
 ⋮
 "thyroid-disease-allhyper"
 "thyroid-disease-allhypo"
 "thyroid-disease-allrep"
 "thyroid-disease-dis"
 "thyroid-disease-sick"
 "thyroid-disease-sick-euthyroid"
 "thyroid-disease-thyroid-0387"
 "trains"
 "wiki4he"

In [40]:
unique(filter(t -> t[:nrow] < 10, combine(groupby(filter(t -> !startswith(t[:method], "Imp-then-Reg 4 - linear"), res), [:dataset, :method]), nrow))[:,:dataset])

8-element Vector{String}:
 "breast-cancer-wisconsin-prognostic"
 "communities-and-crime-2"
 "communities-and-crime"
 "heart-disease-va"
 "horse-colic"
 "mammographic-mass"
 "mlmRev-star"
 "thyroid-disease-allhypo"

In [41]:
unfinished_datasets = filter(t -> t[:nrow] < 10, 
    combine(
        groupby(    combine(groupby(filter(t -> !startswith(t[:method], "Imp-then-Reg 4 - linear"), res), [:dataset, :method]), nrow), 
                    [:dataset]), 
        :nrow => maximum => :nrow))[:,:dataset]
sort([findfirst(i .== dataset_list)-1 for i in unfinished_datasets])

Any[]

In [45]:
unfinished_datasets = filter(t -> t[:nrow] < 10, 
    combine(
        groupby(    combine(groupby(res, [:dataset, :method]), nrow), 
                    [:dataset]), 
        :nrow => minimum => :nrow))[:,:dataset]
sort([findfirst(i .== dataset_list)-1 for i in unfinished_datasets])

8-element Vector{Int64}:
 16
 23
 24
 36
 38
 41
 43
 63

In [44]:
unfinished_datasets = filter(t -> t[:nrow] < 10, 
    combine(
        groupby(    combine(groupby(filter(t -> !startswith(t[:method], "Imp-then-Reg"), res), [:dataset, :method]), nrow), 
                    [:dataset]), 
        :nrow => minimum => :nrow))[:,:dataset]
sort([findfirst(i .== dataset_list)-1 for i in unfinished_datasets])

7-element Vector{Int64}:
 16
 23
 24
 36
 38
 41
 43

In [ ]:
setdiff(), pb_datasets)

# For Synthetic-Data Experiments

In [ ]:
1+1

In [ ]:
setting = prefix*"synthetic_discrete/"
# for y_model in ["linear", "tree", "nn"]
for y_model in ["linear", "nn"]
    for m_model = ["mar", "censoring"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
        
        filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
        res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)

        for subdir = ["all/"]
            filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
            @show filelist

    #         res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
            for i in 1:length(filelist)
                aux = CSV.read(directory*subdir*filelist[i], DataFrame)
                # if any(aux[:,:kMissing] .> 0)
                #     missingproba = unique(aux[aux[:,:pMissing] .> 0,:pMissing])[1]
                #     aux[!,:pMissing] .= missingproba
                # end
                try
                    res = vcat(res, aux)
                catch 
                    println("Error with ", directory*subdir*filelist[i])
                end
            end
        end

        # for subdir = ["xgb/"]
        #     try 
        #         filelist = [f for f in readdir("xgboost/"*directory*subdir) if endswith(f, ".csv")]
        #         for i in 1:length(filelist)
        #             res = vcat(res, CSV.read("xgboost/"*directory*subdir*filelist[i], DataFrame))
        #         end
        #     catch 
        #         println("No XGBoost results for "*directory*subdir)
        #     end
        # end

        res[!,:method] .= map(t -> replace(t, "Encoding as new category" => "Imp-then-Reg 4"), res[:,:method])
        res[!,:method] .= map(t -> replace(t, "Mode impute" => "Imp-then-Reg 5"), res[:,:method])

        # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
        res[!,:method_cat] = map(method_category, res[:,:method])
        res[!,:X_setting] .= "syn_X_"*m_model
        res[!,:Y_setting] .= "syn_Y_"*y_model

        for method in ["Imp-then-Reg 4", "Imp-then-Reg 5"]
            aux = filter(t -> startswith(t[:method], method), res)
            # @show nrow(aux)
            idcols = [:dataset, :X_setting, :Y_setting, :SNR, :k, :pMissing, :splitnum]
            gd = groupby(aux, idcols)

            aux = similar(aux, 0)
            for subdf in gd 
                scoremax = argmax(subdf[:,:score])
                # @show subdf[scoremax,names(aux)]
                push!(aux, subdf[scoremax,names(aux)])
            end
            aux[!,:method] .= method*" - best"

            res = vcat(res, aux)
        end

        CSV.write(directory*"FINAL_results.csv", res)   
    end
end

In [ ]:
setting = prefix*"synthetic/"
files_with_issues = []
# for y_model in ["linear", "tree", "nn"]
for y_model in ["linear", "nn"]
    for m_model = ["mar", "censoring"]
        dir = y_model*"_"*m_model*"/"
        directory = setting*dir
        filelist = [f for f in readdir(directory*"all/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
        res = similar(CSV.read(directory*"all/"*filelist[1], DataFrame),0)
        for subdir = ["all/"]
            filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
    #         res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
            for i in 1:length(filelist)
                try
                    aux = CSV.read(directory*subdir*filelist[i], DataFrame)
                    if any(aux[:,:pMissing] .> 0)
                        missingproba = unique(aux[aux[:,:pMissing] .> 0,:pMissing])[1]
                        aux[!,:pMissing] .= missingproba
                    end
                    res = vcat(res, aux)
                catch 
                    println("Error with ", directory*subdir*filelist[i])
                    push!(files_with_issues, directory*subdir*filelist[i])
                end
            end
        end

        # for subdir = ["xgb/"]
        #     try 
        #         filelist = [f for f in readdir("xgboost/"*directory*subdir) if endswith(f, ".csv")]
        #         for i in 1:length(filelist)
        #             res = vcat(res, CSV.read("xgboost/"*directory*subdir*filelist[i], DataFrame))
        #         end
        #     catch 
        #         println("No XGBoost results for "*directory*subdir)
        #     end
        # end

        # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
        res[!,:method_cat] = map(method_category, res[:,:method])
        res[!,:X_setting] .= "syn_X_"*m_model
        res[!,:Y_setting] .= "syn_Y_"*y_model

        for method in ["Oracle X", "Oracle XM", "Complete Features", "Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4", "Imp-then-Reg 5", "Joint Imp-then-Reg", "Adaptive LR"]
            aux = filter(t -> startswith(t[:method], method), res)

            idcols = [:dataset, :X_setting, :Y_setting, :SNR, :k, :pMissing, :splitnum]
            gd = groupby(aux, idcols)

            aux = similar(aux, 0)
            for subdf in gd 
                scoremax = argmax(subdf[:,:score])
                push!(aux, subdf[scoremax,names(aux)])
            end
            aux[!,:method] .= method*" - best"

            res = vcat(res, aux)
        end
        
        CSV.write(directory*"FINAL_results.csv", res)   
    end
end

In [ ]:
setting

In [ ]:
files_with_issues

In [ ]:
# setting = "synthetic/"
# # for y_model in ["linear", "tree", "nn"]
# for y_model in ["linear", "nn"]
#     for m_model = ["mar", "censoring"]
#         dir = y_model*"_"*m_model*"/"
#         directory = setting*dir
        
#         filelist = [f for f in readdir(directory*"final/") if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
#         res = similar(CSV.read(directory*"final/"*filelist[1], DataFrame),0)

#         for subdir = ["high_n/"]
#             filelist = [f for f in readdir(directory*subdir) if endswith(f, ".csv") && f ∉ ["all_results.csv","all_results_new.csv"]]
#     #         res = similar(CSV.read(directory*subdir*filelist[1], DataFrame),0)
#             for i in 1:length(filelist)
#                 aux = CSV.read(directory*subdir*filelist[i], DataFrame)
#                 if any(aux[:,:pMissing] .> 0)
#                     missingproba = unique(aux[aux[:,:pMissing] .> 0,:pMissing])[1]
#                     aux[!,:pMissing] .= missingproba
#                 end
#                 try
#                     res = vcat(res, aux)
#                 catch 
#                     println("Error with ", directory*subdir*filelist[i])
#                 end
#             end
#         end
#         # filter!(t -> t[:k] > 0, res) #Remove dataset with only a bias term
#         res[!,:method_cat] = map(method_category, res[:,:method])
#         res[!,:X_setting] .= "syn_X_"*m_model
#         res[!,:Y_setting] .= "syn_Y_"*y_model

#         CSV.write(directory*"HIGHN_results.csv", res)   
#     end
# end

In [ ]:
1+1

Sanity check

In [ ]:
df = CSV.read(prefix*"synthetic/linear_mar/FINAL_results.csv", DataFrame) 
df[!,:setting] .= "1 - Lin-MAR"

aux = CSV.read(prefix*"synthetic/linear_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "2 - Lin-NMAR"
df = vcat(df, aux)
 
# aux = CSV.read(prefix*"synthetic/tree_mar/FINAL_results.csv", DataFrame) 
# aux[!,:setting] .= "3 - Tree-MAR"
# df = vcat(df, aux)

# aux = CSV.read(prefix*"synthetic/tree_censoring/FINAL_results.csv", DataFrame) 
# aux[!,:setting] .= "4 - Tree-NMAR"
# df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic/nn_mar/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "5 - NN-MAR"
df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic/nn_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "6 - NN-NMAR"
df = vcat(df, aux)

;

In [ ]:
df = CSV.read(prefix*"synthetic_discrete/linear_mar/FINAL_results.csv", DataFrame) 
df[!,:setting] .= "1 - Lin-MAR"

aux = CSV.read(prefix*"synthetic_discrete/linear_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "2 - Lin-NMAR"
df = vcat(df, aux)
 
# aux = CSV.read(prefix*"synthetic_discrete/tree_mar/FINAL_results.csv", DataFrame) 
# aux[!,:setting] .= "3 - Tree-MAR"
# df = vcat(df, aux)

# aux = CSV.read(prefix*"synthetic_discrete/tree_censoring/FINAL_results.csv", DataFrame) 
# aux[!,:setting] .= "4 - Tree-NMAR"
# df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic_discrete/nn_mar/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "5 - NN-MAR"
df = vcat(df, aux)

aux = CSV.read(prefix*"synthetic_discrete/nn_censoring/FINAL_results.csv", DataFrame) 
aux[!,:setting] .= "6 - NN-NMAR"
df = vcat(df, aux)

In [ ]:
combine(groupby(df, [:dataset, :method, :setting]), nrow)

In [ ]:
unique(combine(groupby(df, [:dataset, :method, :setting]), nrow)[:,:nrow])

In [ ]:
filter( t-> t[:nrow] < 90, combine(groupby(res, [:dataset, :method, :setting]), nrow))

In [ ]:
pbdata = unique(filter( t -> t[:nrow] < 90, combine(groupby(df, [:dataset, :method, :setting]), nrow))[:,[:method, :setting, :dataset]])

In [ ]:
aux = filter( t -> t[:dataset] ∈ pbdata && t[:nrow] < 10, combine(groupby(df, [:dataset, :method, :setting, :pMissing]), nrow))
unique(aux[:,[:dataset, :setting, :pMissing, :nrow]])

In [ ]:
filter(t -> t[:pMissing] == 0.1, unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))

In [ ]:
filter(t -> t[:pMissing] == 0.3, unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))

In [ ]:
filter(t -> startswith(t[:setting], "5"), unique(aux[:,[:dataset, :setting, :pMissing, :nrow]]))